In [1]:
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import make_column_selector, make_column_transformer
from sklearn.metrics import classification_report

from rl_mcts.core.models.FARE import FARE

import pandas as pd
import numpy as np

import random
import torch

# It is done to make the notebook aware of previous classes
import sys
sys.path.append("../")

In [2]:
# Set some random seeds to ensure reproducibility
random.seed(2023)
np.random.seed(2023)
torch.manual_seed(2023)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

In [3]:
# Read adult dataset and preprocess them.
# We remove NaNs values and we split the features from the predictor variable
X = pd.read_csv("../data/adult_score/train.csv")
X.dropna(inplace=True)
y = X.income_target.apply(lambda x: 1 if x=="<=50K" else 0)
X.drop(columns=["income_target", "predicted"], inplace=True)

In [4]:
# Split the dataset into train/test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=2023)

In [5]:
# Build a preprocessing pipeline, which can be used to preprocess
# the elements of the dataset.
cat_selector = make_column_selector(dtype_include=object)
num_selector = make_column_selector(dtype_include=np.number)
preprocessor = make_column_transformer(
    (StandardScaler(), num_selector), (OneHotEncoder(handle_unknown="ignore",sparse=False), cat_selector)
)

In [6]:
# Fit the preprocessor on the training data 
preprocessor.fit(X_train)

ColumnTransformer(transformers=[('standardscaler', StandardScaler(),
                                 <sklearn.compose._column_transformer.make_column_selector object at 0x7fa65633abb0>),
                                ('onehotencoder',
                                 OneHotEncoder(handle_unknown='ignore',
                                               sparse=False),
                                 <sklearn.compose._column_transformer.make_column_selector object at 0x7fa65633ad30>)])

In [7]:
# Fit a simple model over the data
blackbox_model = SVC(class_weight="balanced")
blackbox_model.fit(preprocessor.transform(X_train), y_train)

# Evaluate the model and print the classification report for the two classes
output = blackbox_model.predict(preprocessor.transform(X_test))
print(classification_report(output, y_test))

              precision    recall  f1-score   support

           0       0.87      0.59      0.70       471
           1       0.80      0.95      0.87       832

    accuracy                           0.82      1303
   macro avg       0.84      0.77      0.79      1303
weighted avg       0.83      0.82      0.81      1303



In [8]:
# Filter the training dataset by picking only the examples which are classified negatively by the model
output = blackbox_model.predict(preprocessor.transform(X_train))
X_train["predicted"] = output
X_train = X_train[X_train.predicted == 1]
X_train.drop(columns="predicted", inplace=True)

In [9]:
policy_config= {
        "observation_dim": 103,
        "encoding_dim": 50,
        "hidden_size": 50,
        "model_path": None
    }

environment_config = {
    "class_name": "synthetizer.adult_score.environment.AdultEnvironment",
    "additional_parameters": {
        "preprocessor": preprocessor
    }
}

mcts_config = {
    "exploration": True,
    "number_of_simulations": 10,
    "dir_epsilon": 0.3,
    "dir_noise": 0.3,
    "level_closeness_coeff": 3.0,
    "level_0_penalty": 1.0,
    "qvalue_temperature": 1.0,
    "temperature": 1.3,
    "c_puct": 0.5,
    "gamma": 0.97
}

In [12]:
# Train a FARE model given the previous configurations
model = FARE(blackbox_model, policy_config, environment_config, mcts_config)
model.fit(X_train, max_iter=500, verbose=True)

[*] Iteration 10 / Buffer Size: 2 / 0.039 
[*] Iteration 20 / Buffer Size: 2 / 0.054 
[*] Iteration 30 / Buffer Size: 5 / 0.067 
[*] Iteration 40 / Buffer Size: 11 / 0.070 
[*] Iteration 50 / Buffer Size: 11 / 0.082 
[*] Iteration 60 / Buffer Size: 18 / 0.084 
[*] Iteration 70 / Buffer Size: 20 / 0.076 
[*] Iteration 80 / Buffer Size: 23 / 0.069 
[*] Iteration 90 / Buffer Size: 27 / 0.082 
[*] Iteration 100 / Buffer Size: 30 / 0.093 
[*] Iteration 110 / Buffer Size: 30 / 0.122 
[*] Iteration 120 / Buffer Size: 30 / 0.120 
[*] Iteration 130 / Buffer Size: 37 / 0.128 


KeyboardInterrupt: 

In [13]:
# We save the trained FARE model to disc
model.save("fare.pth")

In [15]:
# We use the model to predict the test data
# We use only the test data which are negatively classified
output = blackbox_model.predict(preprocessor.transform(X_test))
X_test["predicted"] = output
X_test = X_test[X_test.predicted == 1]
X_test.drop(columns="predicted", inplace=True)

In [16]:
# We run inference using FARE
counterfactuals, result, traces, costs = model.predict(X_test[0:10], full_output=True)

In [ ]:
coun